In [1]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

import google.generativeai as genai

from utils import read_config, OracleAgent

from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.llms.openai import OpenAI

import chromadb

c:\Users\sean.chang\AppData\Local\anaconda3\envs\bibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info_path = '.env/info.json'
info = read_config(info_path)

# oracle_agent = OracleAgent(info['DW_conn_info'])

API_KEY= info['gkey']
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-pro')


In [3]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

In [4]:
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embeddings,
)


In [7]:
llm = OpenAI(model='gpt-4o-mini')
query_engine = index.as_query_engine(llm=model)

AttributeError: 'GenerativeModel' object has no attribute 'metadata'

In [6]:
response = query_engine.query("我要看稅前淨利、損益表、資產負債表的數據要看哪張表")

Retrying llama_index.llms.openai.base.OpenAI._chat in 0.36116726540373034 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.5239622209234902 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.

In [ ]:
print(response)